Using the 'binnning_list_of_files.csv' files produced by step3 notebook, this notebook will produced the rebinned files

In [ ]:
import custom_style; 
custom_style.style()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

from ipywidgets.widgets import interact
from ipywidgets import widgets

from pprint import pprint
import pyfits

import datetime
import os
import glob

from plotly.offline import plot, init_notebook_mode, iplot
init_notebook_mode()
import plotly.plotly as py
import plotly.graph_objs as go

from IPython.html import widgets
from IPython.display import display

### Import File 

In [ ]:
root_folder = '/Volumes/My Book Thunderbolt Duo/IPTS/BraggEdge/SNAP/SNAP_August_September_2015/'
file_name = root_folder + 'binning_list_of_files.csv'
df = pd.read_csv(file_name, index_col=0, header=None)

In [ ]:
df.head()

In [ ]:
list_files_to_merge = []
list_times = []
list_temperature = []

for _files in df.loc['list_of_files',:]:
    list_files_to_merge.append(_files.split(';;'))
    
for _time in df.loc['binned_file_time',:]:
    list_times.append(_time)
    
for _temp in df.loc['temperature',:]:
    list_temperature.append(_temp)

### Binning Algorithms

In [ ]:
full_data_name = {}
w = widgets.IntProgress()
w.max = len(list_files_to_merge)
display(w)

for _index, _files in enumerate(list_files_to_merge):
    
    bin_data = []
    
    for _file in _files:

        #retrieve prefix image number   ex: Image0001_0000.fits -> Image0001
        _dirname = os.path.dirname(_file)
        _basename = os.path.basename(_file)
        _basename_splitted = _basename.split('_')
    
        #get full list of image with such a prefix (using glob)
        _full_list = glob.glob(_dirname + '/' + _basename_splitted[0] + '_*.fits')
                
        #store list bin_data.append['file1','file2','file3']
        bin_data.append(_full_list)
        
    #store in full_data dictionary
    # full_data_name{'bin_number': bin_data}
    bin_number  = "%.4d"%_index
    full_data_name[_index] = bin_data
    
    w.value = _index

In [ ]:
print(full_data_name[0])

In [ ]:
fig = plt.figure()
x = list_times
y = list_temperature

plt.step(x, y, where='mid')
plt.xlabel("Time (span)")
plt.ylabel("Temperature(C)")
plt.title("Progress of Rebinned Files Created")






#### Add

####  Median

Not implemented yet

In [ ]:
algorithm = 'median'


#### Mean
Not implemented yet

In [ ]:
algorithm = 'mean'


### Select Output Folder

In [ ]:
output_folder = root_folder + 'rebinned_data/'

### export files

In [ ]:
algorithm = 'add'

def add(data_arrays):
    return np.sum(data_arrays)

data_rebinned = []

for _index, _files in enumerate(list_files_to_merge):
    
    data = read_fits(_files)
    if len(data) == 1:
        data_rebinned.append(data)
    
    _sum_data = data[0]
    for _index_data in range(1, len(data)):
        _sum_data += np.add(_sum_data, data[_index_data])
    data_rebinned.append(_sum_data)

In [ ]:
def read_fits(list_files):
    '''takes a list of files, load them using pyfits and return a list of 
    arrays of data
    '''

    data = []
    
    for _file in list_files:

        hdu_list = pyfits.open(_file)  # fits
        hdu = hdu_list[0]
        _image = hdu.data
        _image = np.asarray(_image)
        data.append(_image)

    return data    

In [ ]:
file_name_prefix = 'Image000'

def make_output_file_name(_files_to_merge, suffix, algorithm='add'):
    '''
    takes the list of files to add and create the output file name 
    
    Paramters:
        * _files_to_merge: list of files to merge
        * suffix: last 4 digits of file names
        * algorithm: (optional) default value 'add'. Name of algorithm used to bin data
            will be used in the new output file name
            
    Return:
        * string file name of the output file
        
    Example:
        _files_to_merge = []'/Users/me/Image0001_00000.fits','/Users/me/Image0002_00000.fits']
        
        will return  'Image00001_Image00002_0000_add.fits
    
    '''
    
    ext = '.fits'
    list_output_file_name = []
    
    for _file in _files_to_merge:
            _basename = os.path.basename(_file)
            _basename_splitted = _basename.split('_')
            list_output_file_name.append(_basename_splitted[0])

    _output_file_name = '_'.join(list_output_file_name)
    return _output_file_name + '_' + algorithm + ext

In [ ]:
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for _index, data in enumerate(data_rebinned):
    _files_to_merge = list_files_to_merge[_index]
    _output_file_name = make_output_file_name(_files_to_merge, algorithm=algorithm)
    _full_output_file_name = output_folder + _output_file_name

    hdu = pyfits.PrimaryHDU(data)
    hdulist = pyfits.HDUList([hdu])
    hdulist.writeto(_full_output_file_name)    